# Basics

Mostly exercises and experiments made during
[Martin Odersky's functional programming course on Coursera](https://www.coursera.org/learn/progfun1)

## (Tail) recursion

In [3]:
import scala.annotation.tailrec

def fibonacci_tail_recursive(n: Long): BigInt = {
// @tailrec enforces method it annotates to be tail recursive
@tailrec def fib(n: Long, acc: BigInt): BigInt = n match {
  case n if n >= 1 => fib(n - 1, acc * n)
  case _ => acc
  }

  fib(n, 1)
}

def fibonacci_not_tail_recursive(n: BigInt): BigInt = n match {
  case n if n > 1 => n * fibonacci_not_tail_recursive(n - 1)
  case _ => 1
}

fibonacci_tail_recursive(1000)

fibonacci_not_tail_recursive(1000)

fibonacci_tail_recursive(100000)

try {
  (3000 to 5000 by 100).map { i =>
    println(s"I is: $i")
    try {
      fibonacci_not_tail_recursive(i)
    } catch {
      case e: StackOverflowError =>
        throw new RuntimeException(
          s"""
             |Stack length that we reached was $i
             |see https://blogs.oracle.com/saas-fusion-app-performance/how-to-set-stack-size-to-overcome-javalangstackoverflowerror")
           """.stripMargin)
    }
  }
} catch {
  case e: RuntimeException =>
    println(e.getMessage)
}

def sum(i: Int, j: Int, f: Int => Int): Int = {
  if (i < j) f(i) + sum(i + 1, j, f)
  else f(j)
}

def sumtr(i: Int, j: Int, f: Int => Int): Int = {
  @tailrec def loop(acc: Int, i: Int, j: Int, f: Int => Int): Int = {
    if(i > j) acc
    else loop(acc + f(i), i + 1, j, f)
  }

  loop(0, i, j, f)
}
sumtr(1, 3, k => k * k)
sum(1, 3, k => k * k)

val params = Tuple3(1,5,(k:Int) => k * k * k)

assert(sum(1, 3, k => k * k) == sumtr(1, 3, k => k * k))
assert(sum(1,5, k => k * k * k) == sumtr(params._1, params._2, params._3))


I is: 3000
I is: 3100
I is: 3200
I is: 3300
I is: 3400
I is: 3500
I is: 3600
I is: 3700
I is: 3800
I is: 3900
I is: 4000
I is: 4100
I is: 4200
I is: 4300
I is: 4400
I is: 4500
I is: 4600
I is: 4700
I is: 4800
I is: 4900
I is: 5000


import scala.annotation.tailrec


defined function fibonacci_tail_recursive
defined function fibonacci_not_tail_recursive
res2_3: BigInt = 402387260077093773543702433923003985719374864210714632543799910429938512398629020592044208486969404800479988610197196058631666872994808558901323829669944590997424504087073759918823627727188732519779505950995276120874975462497043601418278094646496291056393887437886487337119181045825783647849977012476632889835955735432513185323958463075557409114262417474349347553428646576611667797396668820291207379143853719588249808126867838374559731746136085379534524221586593201928090878297308431392844403281231558611036976801357304216168747609675871348312025478589320767169132448426236131412508780208000261683151027341827977704784635868170164365024153691398281264810213092761244896359928705114964975419909342221566832572080821333186116811553615836546984046708975602900950537616475847728421889679646244945160765353408198901385442487984959953319101723355556602139450399736280

#### Use assertions [http://wiki.c2.com/?UseAssertions](http://wiki.c2.com/?UseAssertions)

## Currying and higher order functions

In [36]:
def operationOnInterval(op: (Int, Int) => Int, init: Int)(f: Int=>Int)(i: Int, j: Int): Int = {
  if (i > j) init
  else op(f(i), operationOnInterval(op, init)(f)(i + 1, j))
}

def product(f: Int => Int)(i: Int, j: Int) =
  operationOnInterval((i:Int, j:Int) => i * j, 1)(f)(i, j)

assert(product(k => k * k)(1,4) == 576)
assert(product(k => k)(1,4) == 24)

def isCloseEnough(x: Double, y: Double, tollerance: Double = 0.0001): Boolean = {
  Math.abs(x - y) / y < tollerance
}
assert(! isCloseEnough(1, 1.001, 0.0005))
assert(isCloseEnough(1, 1.001, 0.01))

@tailrec def fixedPoint(f: Double => Double)(init: Double): Double = {
  val next = f(init)
  println(next)
  if(isCloseEnough(next, init)) next
  else fixedPoint(f)(next)
}

fixedPoint(x => 1 + x / 2)(1)

// We can use averageDump whenever we use f, as returned function takes a param (currying)
def averageDump(f: Double => Double)(x: Double): Double = (f(x) + x) / 2

// here we use a fixed point method to define square root
// sqrt is y that y * y = x, so y = x / y - if we find a point where y = x/y, we are close :)

def sqrt(x: Double) = fixedPoint(averageDump((y:Double) => x / y))(1)

sqrt(2)

assert(isCloseEnough(sqrt(2), Math.sqrt(2)))

1.5
1.75
1.875
1.9375
1.96875
1.984375
1.9921875
1.99609375
1.998046875
1.9990234375
1.99951171875
1.999755859375
1.9998779296875
1.5
1.4166666666666665
1.4142156862745097
1.4142135623746899
1.5
1.4166666666666665
1.4142156862745097
1.4142135623746899


defined function operationOnInterval
defined function product
defined function isCloseEnough
defined function fixedPoint
res35_8: Double = 1.9998779296875
defined function averageDump
defined function sqrt
res35_11: Double = 1.4142135623746899

#### Exercise

 1. Define product method (tail recursive)
 1. Define factorial using product